 RAG Pipeline Test-1

In [1]:
import sys
sys.path.append(r'E:\CSE299\chatbot')
from query_Prompt import query_rag

In [2]:
from langchain.tools import Tool
from langchain.agents import create_react_agent
from langchain_community.llms.ollama import Ollama
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.prompts import ChatPromptTemplate



# Custom conversation template for the assistant
CUSTOM_CONVERSATION_TEMPLATE = """
The following is a conversation with a helpful assistant. The assistant is intelligent, knowledgeable, and tries to help the user in the best possible way.

Current Conversation:
{history}

Tools available:
{tools}

Tool names:
{tool_names}

Agent Scratchpad:
{agent_scratchpad}

User: {input}

Assistant:"""

# Tool 1: Physics Knowledge from RAG (retrieval using ChromaDB)
def fetch_from_rag(query: str):
    return query_rag(query)

rag_tool = Tool(
    name="Physics Retrieval",
    func=fetch_from_rag,
    description="Fetch physics-related information from RAG using the local vector database."
)

# Tool 2: Calculation Tool (Simple calculator using Python's eval)
def physics_calculator(expression: str):
    try:
        return eval(expression)
    except Exception as e:
        return str(e)

calc_tool = Tool(
    name="Physics Calculator",
    func=physics_calculator,
    description="Perform calculations relevant to physics problems."
)

# Tool 3: Conversation Memory Tool
def conversation_memory(query: str, chat_history: list):
    # Fetch previous memory for the session from MongoDB
    # conversation_history = fetch_history(session_name)
    return query_rag(query, chat_history) if chat_history else None

memory_tool = Tool(
    name="Conversation Memory",
    func=conversation_memory,
    description="Hold conversation with memory."
)

# Initialize your LLM (Ollama, in this case)
llm = Ollama(model="gemma2:2b")

# Create a list of tools to pass to the agent
tools = [rag_tool, calc_tool, memory_tool]

# Extract tool names for the prompt
tool_names = [tool.name for tool in tools]

# Use create_react_agent for initializing the agent
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=ChatPromptTemplate.from_template(CUSTOM_CONVERSATION_TEMPLATE),
)



In [3]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'history', 'input'], input_types={}, partial_variables={'tools': 'Physics Retrieval(query: str) - Fetch physics-related information from RAG using the local vector database.\nPhysics Calculator(expression: str) - Perform calculations relevant to physics problems.\nConversation Memory(query: str, chat_history: list) - Hold conversation with memory.', 'tool_names': 'Physics Retrieval, Physics Calculator, Conversation Memory'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='\nThe following is a conversation with a helpful assistant. The assistant is intelligent, knowledgeable, and tries to help the user in the best possible way.\n\nCurrent Conversation:\n{history}\n\nTools availa

In [10]:
def run_agent(query: str, chat_history: list):
    # Load previous memory into the agent's memory if available
    # conversation_history = fetch_history(session_name)
    
    # Update the conversation history in the prompt
    if chat_history:
        formatted_history = "\n".join(
            f"{entry['role']}: {entry['content']}" for entry in chat_history
        )
    else:
        formatted_history = ""

    memory_input = f"{formatted_history}\nUser: {query}\nAssistant:"
    # Prepare the agent scratchpad (you can customize this based on your logic)
    agent_scratchpad = ""  # Initialize or load your scratchpad data if needed

    # Run the agent with the user query
    result = agent.invoke({
        "input": query,
        "history": memory_input,
        "tools": "\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
        "tool_names": ", ".join(tool_names),
        "agent_scratchpad": agent_scratchpad,
    })
    print(result) 
    return result


In [6]:
# Sample chat history
chat_history = [
    {"role": "user", "content": "What is gravitational force?"},
    {"role": "bot", "content": "Gravitational force is the force of attraction between two masses..."}
]

In [11]:
def debug_chatbot():

    try:
        # Simulate a user input
        user_input = "My name is Mehar."
        
        # Print current chat history for debugging
        print("Current Chat History:")
        for entry in chat_history:
            print(f"{entry['role']}: {entry['content']}")
        
        # Run the agent with user input and the current chat history
        response = run_agent(user_input, chat_history)
        
        # Print the response from the agent
        print("\nResponse from the chatbot:")
        print(response)
    
    except Exception as e:
        # Print any exceptions that occur during the process
        print(f"An error occurred: {e}")

In [12]:
debug_chatbot()

Current Chat History:
user: What is gravitational force?
bot: Gravitational force is the force of attraction between two masses...
An error occurred: 'intermediate_steps'


In [ ]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'history', 'input'], input_types={}, partial_variables={'tools': 'Physics Retrieval(query: str) - Fetch physics-related information from RAG using the local vector database.\nPhysics Calculator(expression: str) - Perform calculations relevant to physics problems.\nConversation Memory(query: str, chat_history: list) - Hold conversation with memory.', 'tool_names': 'Physics Retrieval, Physics Calculator, Conversation Memory'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='\nThe following is a conversation with a helpful assistant. The assistant is intelligent, knowledgeable, and tries to help the user in the best possible way.\n\nCurrent Conversation:\n{history}\n\nTools availa

In [22]:

from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents.agent_types import AgentType
from langchain.agents import Tool, create_react_agent

In [18]:
pip install numexpr


Note: you may need to restart the kernel to use updated packages.


In [19]:
problem_chain = LLMMathChain.from_llm(llm=Ollama(model="gemma2:2b"))
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only inputmath expressions." 
)

In [25]:
from langchain.agents import create_react_agent

In [56]:
CUSTOM_CONVERSATION_TEMPLATE = """
You are a helpful assistant that can perform reasoning and calculations. You are equipped with the following tools: {tool_names}. 

Current conversation:
{agent_scratchpad}

Question: {input}
"""


In [57]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import create_react_agent

# Define the word problem prompt template
word_problem_template = """You are a reasoning agent tasked with solving 
the user's logic-based questions. Logically arrive at the solution, and be 
factual. In your answers, clearly detail the steps involved and give the 
final answer. Provide the response in bullet points. 
Question: {question} 
Answer:"""

# Define the prompt for the reasoning tool
math_assistant_prompt = PromptTemplate(input_variables=["question"],
                                       template=word_problem_template)

# Create the reasoning chain (reasoning tool)
word_problem_chain = LLMChain(llm=llm,
                              prompt=math_assistant_prompt)

# Define the reasoning tool (for logic-based questions)
word_problem_tool = Tool.from_function(
    name="Reasoning Tool",
    func=word_problem_chain.run,
    description="Useful for when you need to answer logic-based/reasoning questions."
)

# Define the math tool (for arithmetic questions)
math_tool = Tool.from_function(
    name="Calculator",
    func=lambda expression: eval(expression),  # Simple calculator, use a safer eval in production
    description="Useful for answering math questions."
)


In [4]:
from langchain.memory.buffer import ConversationBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from pymongo import MongoClient
import uuid
from datetime import datetime

In [6]:
from langchain.chains.conversation.memory import ConversationEntityMemory

In [8]:
ConversationEntityMemory

ValidationError: 1 validation error for ConversationEntityMemory
llm
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing